# Experiment : Internal variable growth

## Init the Ray client

In [5]:
# Imports
from clases.red_rddas_model import RedRddasModel
import ray
import time
import pandas as pd

# Ray Configurations
# runtime_env = {"working_dir": "/home/carlos/Documents/RESEARCH/SynEstRDDA", "pip": ["requests", "pendulum==2.1.2"]}
# ray.init(address='ray://172.17.163.244:10001', runtime_env=runtime_env , log_to_driver=False)
ray.init(log_to_driver=False, num_cpus=12)

RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.

### Generate the Samples and recollected the indicators

In [ ]:
# Experiment for internal variable growth

#capture the time for all the experiment
v_begin_exp = time.time()

n_experiments = 5
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_of_var_by_rdd_min = 5
    n_of_var_by_rdd_max = 10

    # Fixed Parameters
    n_of_rdds = 8
    n_of_signals_rdd = 2
    n_output_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"

    # List of Result for the Experiments
    l_res_sample = []

    v_n_network = 1
    for n_of_variables_rdd in range(n_of_var_by_rdd_min, n_of_var_by_rdd_max + 1):
        print("Number of Network:", v_n_network)
        print("Number of RDDs:", n_of_rdds)
        print("Number of var:", n_of_variables_rdd)
        print("-------------------------------")


        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        o_rdda_model = RedRddasModel(n_of_rdds, n_of_variables_rdd, n_of_signals_rdd, n_output_variables,
                                       n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        o_rdda_model.generate_rddas(type_network=type_network)

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = RedRddasModel.find_attractors_rddas_ray.remote(o_rdda_model)
        o_rdda_model = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = RedRddasModel.calculation_compatible_pairs.remote(o_rdda_model)
        o_rdda_model = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = RedRddasModel.assembly_attractor_fields_optimized.remote(o_rdda_model)
        o_rdda_model = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_variables": n_of_variables_rdd,
                    "n_rdda_attractors": len(o_rdda_model.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(o_rdda_model.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(o_rdda_model.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_sample.append(res_dict)
        v_n_network = v_n_network + 1

    # Add the sample data to pandas dataframe
    df = pd.DataFrame.from_dict(l_res_sample)
    l_experiments.append(df)
# Capture the time of the experiment
v_end_exp = time.time()
v_time_exp = v_end_exp - v_begin_exp
print("END EXPERIMENT")

Experiment: 1
Number of Network: 1
Number of RDDs: 8
Number of var: 5
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 2
Number of RDDs: 8
Number of var: 6
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 3
Number of RDDs: 8
Number of var: 7
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 4
Number of RDDs: 8
Number of var: 8
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 5
Number of RDDs: 8
Number of var: 9
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 6
Number of RDDs: 8
Number of var: 10
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Experiment: 2
Number of Network: 1
Number of RDDs: 8
Number of var: 5
---------------------------

## Save the experiment data in csv, using pandas Dataframe

In [ ]:
pf_res = pd.concat(l_experiments, keys=range(1,n_experiments+1), names=["n_sample","n_aux"], ignore_index=False)
pf_res.reset_index(drop=True, inplace=True, level=1)
pf_res

In [ ]:
path = "exp2_internal_variable_growth_data.csv"
pf_res.to_csv(path)
print("Experiment saved in:", path)